# Entrega 4 - Redes Neuronales

**Grupo 02:**

| Nombre         | C.I       | Email                        |
|----------------|-----------|------------------------------|
| Santiago Alaniz| 5082647-6 | santiago.alaniz@fing.edu.uy  |
| Bruno De Simone| 4914555-0 | bruno.de.simone@fing.edu.uy  |
| María Usuca    | 4891124-3 | maria.usuca@fing.edu.uy      |



## Objetivo

Considere **[Fashion-MNIST1](https://github.com/zalandoresearch/fashion-mnist/tree/master/data/fashion)**, un conjunto de datos con imágenes de 10 tipos diferentes de artículos de la empresa de vestimenta Zalando, con 60.000 instancias de entrenamiento y 10.000 de evaluación. 

Este laboratorio busca desarrollar y optimizar un clasificador basado en redes neuronales para el dataset `Fashion-MNIST` de Zalando. Inicialmente, se establecerá una arquitectura base de red neuronal feedforward con parámetros específicos.

Posteriormente, se experimentará con tres arquitecturas adicionales para mejorar la clasificación. Al identificar el modelo más prometedor, se aplicarán técnicas de regularización y se comparará su rendimiento con benchmarks existentes. 

Finalmente, se identificarán las imágenes más desafiantes para el clasificador. 

Todo el desarrollo se realizará utilizando `PyTorch`, los resultados y análisis se presentará en esta Jupyter Notebook.

##  Diseño

### Preprocesamiento de datos


### Algoritmo

- i) Defina un red neuronal feedforward base con una capa oculta de 32 unidades y que utilice la función sigmoide como activación, que devuelva, para una imagen de entrada, una distribución de probabilidad sobre las categorías, y permita asignarle la categoría más probable.

### Evaluación

- ii) Utilizando el conjunto de entrenamiento provisto, entrene a la red construida en el paso anterior durante 10 épocas
- iii) Evalúe el rendimiento del clasificador construido sobre un conjunto de validación, utilizando descenso por gradiente estocástico y una tasa de aprendizaje de 0.01.
- iv) Reporte gráficamente la evolución de la pérdida en el conjunto de entrenamiento y de la accuracy sobre el conjunto de validación en función de las iteraciones.

## Experimentación

b) Proponga tres arquitecturas adicionales que busquen mejorar los resultados, modificando la cantidad de unidades, la cantidad de capas ocultas, y/o diferentes funciones de activación. 

Para cada una, evalúe su rendimiento sobre un conjunto de validación, con diferentes valores de tasa de aprendizaje. 

c) A partir del mejor modelo obtenido en b), sugiera y aplique algún mecanimo de regularización y vuelva a evaluar sobre el conjunto de validación, igual que en el paso anterior.

d) Con el mejor modelo obtenido luego de los pasos anteriores, evalúe su performance sobre el conjunto de evaluación utilizando accuracy, precision, recall y medida F1 para cada una de las clases. 

Construya la matriz de confusión. Comente los resultados y compare con los reportados en el sitio del dataset. 

e) Muestre las diez instancias del conjunto de evaluación más “difíciles” para el clasificador construido, utilizando como medida la entropía. Comente los resultados.

## Conclusión

## Bibliografia.

- [Tutorial de PyTorch](https://pytorch.org/tutorials/beginner/basics/intro.html)